# Graphicionado

This notebook reproduces the salient characteristics of the [Graphicionado](https://dl.acm.org/doi/10.5555/3195638.3195707) accelerator.

## Imports

Import the necessary modules.

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "..")

from copy import deepcopy

from src import utils

## Initialization

Initialize the input tensors. Tensor shapes and densities can be modified below. For BFS, use `interval=1`, for SSSP use an `interval=10` (or any integer greater than `1`) when constructing `G_SD`.

In [ ]:
V = 10

density = [1, 0.5]
seed = 0

N = 4
n = 2
m = 2

G_SD = Tensor.fromRandom(rank_ids=["S", "D"], shape=[V, V], seed=seed, density=density, interval=1, name="G")

Visualize the initial graph's adjacency matrix. Skip this step if the graph is large (more than 70 edges).

In [ ]:
displayTensor(G_SD)

## Running Graphicionado with TeAAL/HiFiber

The TeAAL compiler currently only supports basic tensor algebra operations. Specifically, it has three restrictions:
- The **map** operator can only be either multiplication (`*`) or addition (`+`)
- The **reduce** operator can only be addition (`+`)
- The **default** for all outputs can only be `0`

However, graph algorithms require much more variety across all attributes. Therefore, in this notebook, we present a TeAAL specification that *must* be modified to accurately perform breadth-first search (BFS) or single-source shortest path (SSSP).

In this notebook, we do not differentiate between compressed and uncompressed iteration (all iteration is compressed).

### Graphicionado TeAAL Specification

For BFS/SSSP, the property we are trying to assign each vertex is its distance away from the start vertex. For a single iteration of the BFS/SSSP algorithm, we have the following tensors:
- `G`: Graph
- `A0`: Set of active vertices at the start of this iteration (initial frontier)
- `SO`: Graph edges whose sources are in `A0`
- `R`: Potential update to the vertex properties
- `P0`: Set of vertex properties at the start of the iteration
- `P1`: Updated set of vertex properties
- `M`: Set of vertices that were actually updated on this iteration
- `A1`: Set of active vertices at the end of this iteration (updated frontier)

The TeAAL specification for Graphicionado is:
```yaml
einsum:
    declaration:
        G: [D, S]
        A0: [S]
        SO: [D, S]
        R: [V]
        P0: [V]
        P1: [V]
        M: [V]
        A1: [V]
    expressions:
        - SO[d, s] = take(G[d, s], A0[s], 0)
        - R[d] = SO[d, s] * A0[s]
        - P1[v] = R[v] + P0[v]
        - M[v] = P1[v] + s * P0[v]
        - A1[v] = take(M[v], P1[v], 1)
mapping:
    rank-order:
        G: [S, D]
    partitioning:
        SO:
            D: [uniform_shape(N)]
            S: [uniform_shape(N), uniform_shape(m)]
        R:
            D: [uniform_shape(N), uniform_shape(n)]
            S: [uniform_shape(N)]
            V: [follow(D)]
        P1:
            V: [uniform_shape(N), uniform_shape(n)]
        M:
            V: [uniform_shape(N), uniform_shape(n)]
        A1:
            V: [uniform_shape(N), uniform_shape(n)]
    loop-order:
        SO: [D1, S2, S1, S0, D0]
        R: [V2, S1, S0, V1, V0]
        P1: [V2, V1, V0]
        M: [V2, V1, V0]
        A1: [V2, V1, V0]
    spacetime:
        SO:
            space: [S0.coord]
            time: [D1, S2, S1, D0]
            opt: slip
        R:
            space: [V0.coord]
            time: [V2, S1, S0, V1]
            opt: slip
        P1:
            space: [V0.coord]
            time: [V2, V1]
            opt: slip
        M:
            space: [V0.coord]
            time: [V2, V1]
            opt: slip
        A1:
            space: [V0.coord]
            time: [V2, V1]
            opt: slip
```

### Per-Run Initialization

Create the tensor of active vertices and the tensor of property values. The `start_vertex` can be modified below.

In [ ]:
start_vertex = 0

A0_S = Tensor.fromFiber(rank_ids=["S"], fiber=Fiber([start_vertex], [0]), default=float("inf"), name="A0")
P0_V = Tensor.fromFiber(rank_ids=["V"], fiber=Fiber([start_vertex], [0], default=float("inf")), default=float("inf"), name="P0")

### Modified HiFiber Ouput

Below is the HiFiber output obtained by compiling the above TeAAL specification (minus the `spacetime` stamp). It has been modified to actually perform BFS/SSSP.

In general, the changes are as follows:
- Multiplication (`*`) is remapped to addition (`+`)
- Addition (`+`) is remapped to minimum (`min`)
- Subtraction (`-` or `+ s *`) is remapped to not equal (`!=`)

To match this change, the default values for the following tensors need to be manually updated:
- `R`: `float("inf")`
- `M`: `False`
- `A1`: `float("inf")`

Additionally, Einsum notation is static single assigment (SSA), but the vector of properties is *updated* from iteration to iteration. So, instead of creating a new tensor `P1`, we must copy `P0` into `P1`.

Finally, we add a loop, which continues as long as there are active vertices.

The inline comments describe the changes.

In [ ]:
# The TeAAL specification is for a single iteration, to perform the full BFS/SSSP, loop until the set of active vertices is empty
while len(A0_S.getRoot()) > 0:

    # Track progress by displaying vertex distance seen so far
    P0_V.setName("P")
    displayTensor(P0_V)

    # Modified HiFiber
    SO_D1S2S1S0D0 = Tensor(rank_ids=["D1", "S2", "S1", "S0", "D0"], name="SO")
    tmp0 = G_SD
    tmp1 = tmp0.splitUniform(N, depth=0)
    tmp2 = tmp1.splitUniform(m, depth=1)
    G_S2S1S0D = tmp2
    G_S2S1S0D.setRankIds(rank_ids=["S2", "S1", "S0", "D"])
    tmp3 = G_S2S1S0D
    tmp4 = tmp3.splitUniform(N, depth=3)
    G_S2S1S0D1D0 = tmp4
    G_S2S1S0D1D0.setRankIds(rank_ids=["S2", "S1", "S0", "D1", "D0"])
    tmp5 = A0_S
    tmp6 = tmp5.splitUniform(N, depth=0)
    tmp7 = tmp6.splitUniform(m, depth=1)
    A0_S2S1S0 = tmp7
    A0_S2S1S0.setRankIds(rank_ids=["S2", "S1", "S0"])
    so_d1 = SO_D1S2S1S0D0.getRoot()
    G_D1S2S1S0D0 = G_S2S1S0D1D0.swizzleRanks(rank_ids=["D1", "S2", "S1", "S0", "D0"])
    g_d1 = G_D1S2S1S0D0.getRoot()
    a0_s2 = A0_S2S1S0.getRoot()
    for d1, (so_s2, g_s2) in so_d1 << g_d1:
        for s2, (so_s1, (g_s1, a0_s1)) in so_s2 << (g_s2 & a0_s2):
            for s1, (so_s0, (g_s0, a0_s0)) in so_s1 << (g_s1 & a0_s1):
                for s0, (so_d0, (g_d0, a0_val)) in so_s0 << (g_s0 & a0_s0):
                    for d0, (so_ref, g_val) in so_d0 << g_d0:
                        so_ref <<= g_val
    tmp8 = SO_D1S2S1S0D0
    tmp9 = tmp8.swizzleRanks(rank_ids=["D1", "D0", "S2", "S1", "S0"])
    tmp10 = tmp9.mergeRanks(depth=0, levels=1, coord_style="absolute")
    tmp11 = tmp10.mergeRanks(depth=1, levels=2, coord_style="absolute")
    tmp11.setRankIds(rank_ids=["D", "S"])
    SO_DS = tmp11
    # Custom default
    R_V2V1V0 = Tensor(rank_ids=["V2", "V1", "V0"], name="R", default=float("inf"))
    tmp12 = SO_DS
    tmp13 = tmp12.splitUniform(N, depth=1)
    SO_DS1S0 = tmp13
    SO_DS1S0.setRankIds(rank_ids=["D", "S1", "S0"])
    tmp14 = SO_DS1S0
    tmp15 = tmp14.splitUniform(N, depth=0)
    tmp16 = tmp15.splitUniform(n, depth=1)
    SO_D2D1D0S1S0 = tmp16
    SO_D2D1D0S1S0.setRankIds(rank_ids=["D2", "D1", "D0", "S1", "S0"])
    tmp17 = A0_S
    tmp18 = tmp17.splitUniform(N, depth=0)
    A0_S1S0 = tmp18
    A0_S1S0.setRankIds(rank_ids=["S1", "S0"])
    r_v2 = R_V2V1V0.getRoot()
    SO_D2S1S0D1D0 = SO_D2D1D0S1S0.swizzleRanks(rank_ids=["D2", "S1", "S0", "D1", "D0"])
    so_d2 = SO_D2S1S0D1D0.getRoot()
    a0_s1 = A0_S1S0.getRoot()
    for v2, (r_v1, so_s1) in r_v2 << so_d2.project(trans_fn=lambda d2: d2):
        for s1, (so_s0, a0_s0) in so_s1 & a0_s1:
            for s0, (so_d1, a0_val) in so_s0 & a0_s0:
                inputs_v1 = Fiber.fromLazy(so_d1.project(trans_fn=lambda d1: d1))
                for v1_pos, (v1, (r_v0, so_d0)) in enumerate(r_v1 << so_d1.project(trans_fn=lambda d1: d1)):
                    if v1_pos == 0:
                        v0_start = 0
                    else:
                        v0_start = v1
                    if v1_pos + 1 < len(inputs_v1):
                        v0_end = inputs_v1.getCoords()[v1_pos + 1]
                    else:
                        v0_end = V
                    for v0, (r_ref, so_val) in r_v0 << so_d0.project(trans_fn=lambda d0: d0, interval=(v0_start, v0_end)):
                        # + => min, * => +
                        r_ref <<= min(r_ref, so_val + a0_val)
    tmp19 = R_V2V1V0
    tmp20 = tmp19.mergeRanks(depth=0, levels=2, coord_style="absolute")
    tmp20.setRankIds(rank_ids=["V"])
    R_V = tmp20
    tmp21 = R_V
    tmp22 = tmp21.splitUniform(N, depth=0)
    tmp23 = tmp22.splitUniform(n, depth=1)
    R_V2V1V0 = tmp23
    R_V2V1V0.setRankIds(rank_ids=["V2", "V1", "V0"])
    tmp24 = P0_V
    tmp25 = tmp24.splitUniform(N, depth=0)
    tmp26 = tmp25.splitUniform(n, depth=1)
    P0_V2V1V0 = tmp26
    P0_V2V1V0.setRankIds(rank_ids=["V2", "V1", "V0"])
    # P0 and P1 are actually the same tensor
    P1_V2V1V0 = deepcopy(P0_V2V1V0)
    p1_v2 = P1_V2V1V0.getRoot()
    r_v2 = R_V2V1V0.getRoot()
    p0_v2 = P0_V2V1V0.getRoot()
    for v2, (p1_v1, (_, r_v1, p0_v1)) in p1_v2 << (r_v2 | p0_v2):
        for v1, (p1_v0, (_, r_v0, p0_v0)) in p1_v1 << (r_v1 | p0_v1):
            for v0, (p1_ref, (_, r_val, p0_val)) in p1_v0 << (r_v0 | p0_v0):
                # + => min
                p1_ref <<= min(r_val, p0_val)
    tmp27 = P1_V2V1V0
    tmp28 = tmp27.mergeRanks(depth=0, levels=2, coord_style="absolute")
    tmp28.setRankIds(rank_ids=["V"])
    P1_V = tmp28
    # Custom default
    M_V2V1V0 = Tensor(rank_ids=["V2", "V1", "V0"], name="M", default=False)
    tmp29 = P1_V
    tmp30 = tmp29.splitUniform(N, depth=0)
    tmp31 = tmp30.splitUniform(n, depth=1)
    P1_V2V1V0 = tmp31
    P1_V2V1V0.setRankIds(rank_ids=["V2", "V1", "V0"])
    tmp32 = P0_V
    tmp33 = tmp32.splitUniform(N, depth=0)
    tmp34 = tmp33.splitUniform(n, depth=1)
    P0_V2V1V0 = tmp34
    P0_V2V1V0.setRankIds(rank_ids=["V2", "V1", "V0"])
    m_v2 = M_V2V1V0.getRoot()
    p1_v2 = P1_V2V1V0.getRoot()
    p0_v2 = P0_V2V1V0.getRoot()
    for v2, (m_v1, (_, p1_v1, p0_v1)) in m_v2 << (p1_v2 | p0_v2):
        for v1, (m_v0, (_, p1_v0, p0_v0)) in m_v1 << (p1_v1 | p0_v1):
            for v0, (m_ref, (_, p1_val, p0_val)) in m_v0 << (p1_v0 | p0_v0):
                # - => !=
                m_ref <<= p1_val != p0_val
    tmp35 = M_V2V1V0
    tmp36 = tmp35.mergeRanks(depth=0, levels=2, coord_style="absolute")
    tmp36.setRankIds(rank_ids=["V"])
    M_V = tmp36
    # Custom default
    A1_V2V1V0 = Tensor(rank_ids=["V2", "V1", "V0"], name="A1", default=float("inf"))
    tmp37 = M_V
    tmp38 = tmp37.splitUniform(N, depth=0)
    tmp39 = tmp38.splitUniform(n, depth=1)
    M_V2V1V0 = tmp39
    M_V2V1V0.setRankIds(rank_ids=["V2", "V1", "V0"])
    tmp40 = P1_V
    tmp41 = tmp40.splitUniform(N, depth=0)
    tmp42 = tmp41.splitUniform(n, depth=1)
    P1_V2V1V0 = tmp42
    P1_V2V1V0.setRankIds(rank_ids=["V2", "V1", "V0"])
    a1_v2 = A1_V2V1V0.getRoot()
    m_v2 = M_V2V1V0.getRoot()
    p1_v2 = P1_V2V1V0.getRoot()
    for v2, (a1_v1, (m_v1, p1_v1)) in a1_v2 << (m_v2 & p1_v2):
        for v1, (a1_v0, (m_v0, p1_v0)) in a1_v1 << (m_v1 & p1_v1):
            for v0, (a1_ref, (m_val, p1_val)) in a1_v0 << (m_v0 & p1_v0):
                a1_ref <<= p1_val
    tmp43 = A1_V2V1V0
    tmp44 = tmp43.mergeRanks(depth=0, levels=2, coord_style="absolute")
    tmp44.setRankIds(rank_ids=["V"])
    A1_V = tmp44

    # Prepare for the next iteration
    P0_V = P1_V
    A0_S = A1_V

### Check Results

Check that generated code computes the correct result.

**Note**: Should be used after running the kernel (above cell).

In [ ]:
utils.check_bfs_sssp(G_SD, start_vertex, P1_V)